In [39]:
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from collections import Counter
import string

# Ensure the necessary NLTK datasets are downloaded
nltk.download('words')
nltk.download('punkt')

# Load the words corpus
word_list = words.words()

# Create a frequency distribution of words from the corpus
WORDS = Counter(word_list)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts    = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def autocorrect_text(text):
    tokens = word_tokenize(text)
    corrected_tokens = []
    for word in tokens:
        if word.lower() not in string.punctuation:
            suggestions = candidates(word.lower())
            corrected_word = max(suggestions, key=P)
            # Preserve the original capitalization
            if word[0].isupper():
                corrected_word = corrected_word.capitalize()
            corrected_tokens.append(corrected_word)
        else:
            corrected_tokens.append(word)
    corrected_text = ' '.join(corrected_tokens)
    return corrected_text

# Example usage
input_text = "This is a smple text with sme spellling eroors."
corrected_text = autocorrect_text(input_text)
print(f"Original text: {input_text}")
print(f"Corrected text: {corrected_text}")


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sarav\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sarav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original text: This is a smple text with sme spellling eroors.
Corrected text: This is a simple text with some spelling error .
